# <span style="font-family: 'Computer Modern'; font-size: 36pt; font-weight: bold;">Quantum Convolutional Neural Network (QCNN) Using *PennyLane*</span>
###### <span style="font-family: 'Computer Modern'; font-size: 10pt; font-weight: bold;">© 2024 Sean Chisholm</span>

#### <span style="font-family: 'Computer Modern'; font-size: 20pt; font-weight: bold;">Introduction:</span> 
<span style="font-family: 'Computer Modern'; font-size: 15pt;">This Jupyter Notebook demonstrates the implementation and analysis of a custom Quantum Convolutional Neural Network (QCNN) using PennyLane.</span>
<span style="font-family: 'Computer Modern'; font-size: 15pt;">The goal is to apply quantum computing techniques to enhance classical machine learning models for image classification tasks.</span>

#### <span style="font-family: 'Computer Modern'; font-size: 20pt; font-weight: bold;">Objective:</span> 
<span style="font-family: 'Computer Modern'; font-size: 15pt;">The main objective is to create and evaluate a QCNN for classifying the MNIST dataset, showcasing the implementation of quantum convolutional and pooling layers.</span>

#### <span style="font-family: 'Computer Modern'; font-size: 20pt; font-weight: bold;">Functionality:</span> 
<span style="font-family: 'Computer Modern'; font-size: 15pt;">*TODO*</span>

#### <span style="font-family: 'Computer Modern'; font-size: 20pt; font-weight: bold;">Dependencies:</span> 
<span style="font-family: 'Computer Modern'; font-size: 15pt;">This project uses the following libraries:</span>
<span style="font-family: 'Computer Modern'; font-size: 15pt;">PennyLane,</span>
<span style="font-family: 'Computer Modern'; font-size: 15pt;">NumPy,</span>
<span style="font-family: 'Computer Modern'; font-size: 15pt;">Matplotlib,</span>
<span style="font-family: 'Computer Modern'; font-size: 15pt;">SciPy</span>

#### <span style="font-family: 'Computer Modern'; font-size: 20pt; font-weight: bold;">Structure:</span> 
<span style="font-family: 'Computer Modern'; font-size: 15pt;">*TODO*</span>

#### <span style="font-family: 'Computer Modern'; font-size: 20pt; font-weight: bold;">References and Further Reading:</span> 
<span style="font-family: 'Computer Modern'; font-size: 15pt;">1. [Source 1 URL]</span>
<span style="font-family: 'Computer Modern'; font-size: 15pt;">2. [Source 2 URL]</span>
<span style="font-family: 'Computer Modern'; font-size: 15pt;">3. [Source 3 URL]</span>

#### <span style="font-family: 'Computer Modern'; font-size: 20pt; font-weight: bold;">Acknowledgements:</span> 
<span style="font-family: 'Computer Modern'; font-size: 15pt;">This research was conducted under the Laboratory of Particle Physics and Cosmology at Harvard University, led by Professor Carlos Arguelles-Delgado.</span>

***

<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Imports_:</span>

In [1]:
# QML-Based Imports:
import pennylane as qml
from pennylane import numpy as np
import torch

# Data Recognition/Visualization Imports:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)
from tqdm import tqdm
import csv

# Data Analysis Imports:
import random # for generating random data
from sklearn import datasets # for data modeling/constructing
from sklearn.model_selection import train_test_split # for data modeling/constructing
from sklearn.svm import SVC # for data training

# PennyLane Demo-Based Imports (MNIST Data):
from pennylane.templates import RandomLayers

In [2]:
# TorchVision:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
# QML-Drawing-Based Imports:
import matplotlib.patches as patches

# QML Data-Based Imports:
from sklearn.preprocessing import OneHotEncoder # for encoding training and testing data labels

# NEW Imports:
import copy
import sys
from scipy.linalg import expm

***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">FUNCTIONS / CLASSES</span>

In [4]:
# Gell Mann Matrix Function:

def generate_gell_mann(order):
    """
    Generates a list of np.arrays which represent Gell Mann matrices of order 'order'.
    eg: order = 2
    gm_matrices = [ [[0,  1],
                             [1,  0]] ,

                            [[0, -i]
                             [i,  0]] ,

                            [[1,  0],
                             [0, -1]] ]
    """
    gm_matrices = []  # Initialize an empty list to store matrices:
    for k in range(order):
        j = 0
        while j < k:
            # Generate symmetric and anti-symmetric matrices:
            sym = b_mat(j, k, order) + b_mat(k, j, order)  # Create a symmetric matrix:
            # Create an anti-symmetric matrix:
            anti_sym = complex(0.0, -1.0) * (b_mat(j, k, order) - b_mat(k, j, order))
            gm_matrices.append(sym)  # Append symmetric matrix to list:
            gm_matrices.append(anti_sym)  # Append anti-symmetric matrix to list:
            j += 1

        if k < (order - 1):
            n = k + 1
            coeff = np.sqrt(2 / (n * (n + 1)))  # Calculate coefficient for diagonal matrix:

            sum_diag = b_mat(0, 0, order)  # Start with first diagonal element:
            for i in range(1, k + 1):
                sum_diag += b_mat(i, i, order)  # Sum up to k-th diagonal element:

            diag_mat = coeff * (sum_diag - n * (b_mat(k + 1, k + 1, order)))  # Create diagonal matrix:
            gm_matrices.append(diag_mat)  # Append diagonal matrix to list:

    # Return list of Gell-Mann matrices:
    return gm_matrices

In [5]:
# Convolutional Operator Function:

def get_conv_op(mats, params):
    """
    Parametrizes the convolutional operator according to Gell-Mann matrices scaled by trainable parameters,
    this method generates the relevant applicable operator.
    """
    # Initialize a zero matrix with same shape as Gell-Mann matrices:
    final = np.zeros(mats[0].shape, dtype=np.complex128)
    for mat, param in zip(mats, params):  # Iterate over pairs of matrices and parameters:
        final += param * mat  # Accumulate weighted sum of matrices:

    # Return matrix exponential of final matrix:
    return expm(complex(0, -1) * final)

In [6]:
# Basis Matrix Function:

def b_mat(i, j, n):
    """
    Generates an n x n matrix of 0s with the i,j th entry is a one.
    This is the i,j th basis vector on the space of n x n real matricies

    :param i: int, row index (must be < n)
    :param j: int, column index (must be < n)
    :param n: int, dimension of the matrices
    :return: np.array of floats, shape (n,n)
    """
    basis_matrix = np.zeros((n, n), dtype=np.float32)
    basis_matrix[i, j] = 1.0

    return basis_matrix

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">DATA</span>

<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Loading / Processing_:</span>

In [7]:
# NOTE: This function uses TensorFlow to import the MNIST dataset, and due to environment errors, it is currently
# not in use. Intead, we import the MNIST dataset using TorchVision.

# Using TensorFLow (Currently Not in Use):
def load_mnist_tf():
    """
    Load the MNIST dataset and return training and testing images and labels, using TensorFlow.
    
    Returns:
        train_images (numpy.ndarray): Training images.
        train_labels (numpy.ndarray): Training labels.
        test_images (numpy.ndarray): Testing images.
        test_labels (numpy.ndarray): Testing labels.
    """
    # Load MNIST dataset using TensorFlow:
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    return train_images, train_labels, test_images, test_labels

In [8]:
# Define normalization transform:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download and load train data:
#------------------------------------------------------
# NOTE: Adjust directory based on your user directory.
#------------------------------------------------------
trainset = datasets.MNIST(root='/Users/seanchisholm/NeutrinosSummer24/QML_Summer-2024/QML-JUNE/DATA', train=True,
                          transform=transform, download=True)
trainloader = DataLoader(trainset, batch_size=1000, shuffle=True)

# Download and load test data:
#------------------------------------------------------
# NOTE: Adjust directory based on User
#------------------------------------------------------
testset = datasets.MNIST(root='/Users/seanchisholm/NeutrinosSummer24/QML_Summer-2024/QML-JUNE/DATA', train=False,
                         transform=transform, download=True)
testloader = DataLoader(testset, batch_size=1000, shuffle=False)

# Extract data from loaders:
train_images, train_labels = next(iter(trainloader))
test_images, test_labels = next(iter(testloader))

# Print relevant shapes and types V1:
print("Shape of train_images:", train_images.shape)
print("Shape of train_labels:", train_labels.shape)
print("Shape of test_images:", test_images.shape)
print("Type of train_images:", type(train_images))
print("Type of train_labels:", type(train_labels))

Shape of train_images: torch.Size([1000, 1, 28, 28])
Shape of train_labels: torch.Size([1000])
Shape of test_images: torch.Size([1000, 1, 28, 28])
Type of train_images: <class 'torch.Tensor'>
Type of train_labels: <class 'torch.Tensor'>


<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Reduction_:</span>

In [9]:
# Reduce dataset size:
n_train = 500
n_test = 100
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Multi-Classification -> Binary Classification:
#---------------------------------------------------------------------------
# Select indices for classes 0 and 1 in train_labels:
train_idx = np.append(np.where(train_labels.numpy() == 0)[0][:n_train//2],
                      np.where(train_labels.numpy() == 1)[0][:n_train//2])

# Select indices for classes 0 and 1 in test_labels:
test_idx = np.append(np.where(test_labels.numpy() == 0)[0][:n_test//2],
                     np.where(test_labels.numpy() == 1)[0][:n_test//2])

# Convert indices to PyTorch tensors:
train_idx = torch.tensor(train_idx, dtype=torch.long)
test_idx = torch.tensor(test_idx, dtype=torch.long)

# Filter training and testing data based on 'train_idx' and 'test_idx':
train_images = train_images[train_idx]
test_images = test_images[test_idx]

# Filter labels based on based on 'train_idx' and 'test_idx':
train_labels = train_labels[train_idx]
test_labels = test_labels[test_idx]
#---------------------------------------------------------------------------

# Normalize pixel values within [0,1]:
train_images = train_images / 255.0
test_images = test_images / 255.0

# Print relevant shapes and types V2:
print("Shape of train_images:", train_images.shape)
print("Shape of train_labels:", train_labels.shape)
print("Shape of test_images:", test_images.shape)
print("Shape of test_labels:", test_labels.shape)
print("Type of train_images:", type(train_images))
print("Type of train_labels:", type(train_labels))

Shape of train_images: torch.Size([108, 1, 28, 28])
Shape of train_labels: torch.Size([108])
Shape of test_images: torch.Size([22, 1, 28, 28])
Shape of test_labels: torch.Size([22])
Type of train_images: <class 'torch.Tensor'>
Type of train_labels: <class 'torch.Tensor'>


<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Flattening_:</span>

In [10]:
# Flatten images:
train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

# Print relevant shapes and types V3:
print("Shape of train_images:", train_images.shape)
print("Shape of train_labels:", train_labels.shape)
print("Shape of test_images:", test_images.shape)
print("Type of train_images:", type(train_images))
print("Type of train_labels:", type(train_labels))

Shape of train_images: torch.Size([108, 784])
Shape of train_labels: torch.Size([108])
Shape of test_images: torch.Size([22, 784])
Type of train_images: <class 'torch.Tensor'>
Type of train_labels: <class 'torch.Tensor'>


***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">QCNN MODEL</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Layers_:</span>

In [11]:
# Convolutional Layer Function:
def quantum_conv_layer(weights):
    """
    Applies a quantum convolutional layer.
    """
    n_qubits = 10  # Number of qubits is second dimension of weights
    
    if np.ndim(weights) == 0:
        weights = np.array([weights])
    
    for i in range(0, n_qubits, 2):
        # First rotation on second qubit of pair:
        qml.RZ(-np.pi / 2, wires=i + 1)
        
        # First set of rotations on first qubit of pair:
        qml.RZ(weights[0], wires=i)
        qml.RY(weights[0], wires=i + 1)
        
        # First CNOT gate:
        qml.CNOT(wires=[i, i + 1])
        
        # Second set of rotations on second qubit of pair:
        qml.RY(weights[0], wires=i + 1)
        
        # Second CNOT gate:
        qml.CNOT(wires=[i + 1, i])
        
        # Second rotation on first qubit of pair:
        qml.RZ(np.pi / 2, wires=i)

In [12]:
# Pooling Layer Function:
def pennylane_pool_layer(weights):
    """
    Applies two-qubit pooling operation to inputted qubits, including controlled rotation based on 
    measurement.
    """
    n_qubits = 10  # Determine number of qubits from weights shape:

    # Generate Gell-Mann matrices for 2-D space (single qubit operators):
    pool_operators = generate_gell_mann(2)  # NEW FUNC
    
    # Loop over active qubits in pairs # FOR PAIRS OF QUBITS
    for i in range(0, n_qubits, 2):  # FOR PAIRS OF QUBITS
        q1 = i        # First qubit:
        q2 = i + 1    # Second qubit:

        # Extract weights for current qubit pair:
        new_weights = weights[i]

        # Get convolutional operators V1 and V2 from pool operators and weights:
        v1 = get_conv_op(pool_operators, new_weights[0])  # Use first set of weights for V1  # NEW FUNC
        v2 = get_conv_op(pool_operators, new_weights[1])  # Use second set of weights for V2  # NEW FUNC

        # Apply Hadamard gate to first qubit:
        qml.Hadamard(wires=q1)
        
        # Apply first controlled unitary operation:
        qml.ControlledQubitUnitary(v1, control_wires=[q1], wires=[q2])
        
        # Apply Hadamard gate to second qubit:
        qml.Hadamard(wires=q2)
        
        # Apply second controlled unitary operation:
        qml.ControlledQubitUnitary(v2, control_wires=[q2], wires=[q1])

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Circuit_:</span>

In [13]:
# Define device(s):
n_qubits = 10  # Number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# QCNN:
@qml.qnode(dev, interface='autograd')
def q_conv_circuit(params, x):
    """
    Defines a quantum circuit for training. This circuit encodes the input features using RX gates, applies
    two quantum convolutional layers interleaved with a pooling layer, and measures the expectation value of
    Pauli-Z on the first qubit.
    """
    # Apply Amtplitude Embedding:
    qml.AmplitudeEmbedding(x, wires=range(n_qubits), normalize=True)
    
    # Apply first convolutional layer function, pass 'params' as argument:
    quantum_conv_layer(params)

    # Apply pooling layer, pass 'params' as argument:
    pennylane_pool_layer(params)
    
    # Return measure of the PauliZ expectation value:
    return [qml.expval(qml.PauliZ(wire)) for wire in range(0, n_qubits, 2)]

***

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">TRAINING</span>

<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Weights_:</span>

In [14]:
def transform_params(params):
    
    params_complex = params.astype(np.complex128) # Convert to 'complex128'
    params = torch.tensor(params_complex) # Convert to TorchVision tensor
    
    return params

In [15]:
# Transform weights into appropriate broadcasting form:
def broadcast_params(params):
    
    # Initialize parameters:
    n_qubits = 10  # Number of qubits
    
    # Flatten images:
    params_flat = params.reshape(-1)
    # print(params_flat[0])
    
    opt_length = 2**n_qubits
    
    # Pad with zeros to match opt_length:
    if len(params_flat) < opt_length:
        params_flat = np.pad(params_flat, (0, opt_length - len(params_flat)), mode='constant', constant_values=0.0)
    
    params = np.array(params_flat)
    # params = transform_params(params)
    
    return params

# Initialize weights:
qcnn_weights = np.random.uniform(0, np.pi, size=(10, n_qubits, 3))
qcnn_weights = broadcast_params(qcnn_weights)

# Print relevant shapes and types V5:
print(f"Shape of Example Parameter: {qcnn_weights[0]}")
print("Shape of 'params':", qcnn_weights.shape)
print("Type of 'params':", type(qcnn_weights))

Shape of Example Parameter: 0.272673230304305
Shape of 'params': (1024,)
Type of 'params': <class 'pennylane.numpy.tensor.tensor'>


<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Cost_:</span>

In [16]:
# Mean Squared Error (MSE) Function:
def mse_cost(params, x, y):
    """
    Computes the Mean Squared Error (MSE) cost function.
    """
    predictions = np.array([q_conv_circuit(params, xi) for xi in x])
    return np.mean((predictions - y) ** 2)

<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Optimization_:</span>

In [17]:
# NOTE: Currently Not Operative

# Training and Testing Data Instantiation:
#------------------------------------------------------------------------
# x_train = np.array([data[0] for data in train_images])
# x_test = np.array([data[0] for data in test_images])
x_train = np.array(train_images)
x_test = np.array(test_images)

y_train = np.array(train_labels)
y_test = np.array(test_labels)

# Pad x_train and x_test with zeros to desired shape (,1024):
x_train = np.pad(x_train, ((0, 0), (0, 1024 - x_train.shape[1])), mode='constant')
x_test = np.pad(x_test, ((0, 0), (0, 1024 - x_test.shape[1])), mode='constant')

# Print relevant shapes and types V6:
print(f"x_train shape: {x_train.shape}, dtype: {x_train.dtype}")
print(f"x_test shape: {x_test.shape}, dtype: {x_test.dtype}")
print(f"y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"y_test shape: {y_test.shape}, dtype: {y_test.dtype}")
#------------------------------------------------------------------------

# Define relevant cost function (Using MSE for now):
cost = mse_cost # sets cost func to mean squared error

# Parameters for optimization:
learning_rate = 0.01
num_steps = 100
batch_size = 32

# Initialize parameters:
n_qubits = 10  # Number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Stochastic Gradient Descent Function:
def stoch_grad_descent(params, x, y, learning_rate, batch_size):
    # Shuffle Data:
    permutation = np.random.permutation(len(x))
    x = x[permutation]
    y = y[permutation]

    # Process each Batch:
    for i in range(0, len(x), batch_size):
        x_batch = x[i:i + batch_size]
        y_batch = y[i:i + batch_size]

        # Compute Gradient:
        grad_cost = qml.grad(mse_cost, argnum=0)
        gradients = grad_cost(params, x_batch, y_batch)

        # Update Parameters:
        params = params - learning_rate * gradients

    return params

step_size = 5

# Training Loop:
for step in range(num_steps):
    qcnn_weights = stoch_grad_descent(qcnn_weights, x_train, y_train, learning_rate, batch_size)
    if step % step_size == 0:
        loss = mse_cost(qcnn_weights, x_train, y_train)
        print(f"Step {step}: cost = {loss}")

# Evaluate optimization on sample test data:
predictions = np.array([q_conv_circuit(qcnn_weights, xi) for xi in x_test])

x_train shape: (108, 1024), dtype: float32
x_test shape: (22, 1024), dtype: float32
y_train shape: (108,), dtype: int64
y_test shape: (22,), dtype: int64


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

***

<span style="font-family: 'Computer Modern'; font-size: 16pt;">_Accuracy_:</span>

In [ ]:
# Convert continuous predictions to class labels (-1 or 1) depending on value distance:
predicted_labels = np.where(predictions >= 0, 1, -1)

# Calculate accuracy:
def calculate_accuracy(y_true, y_pred):
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions
    return accuracy

# Compute accuracy on test dataset:
accuracy = calculate_accuracy(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

***

## <span style="font-family: 'Computer Modern'; font-size: 24pt; font-weight: bold;">APPENDIX: _More Circuits, Gates, Drawings, and Analysis_</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Parameter Re-Initialization_:</span>

In [21]:
# Analyzing Weights Shape/Size/Behavior:

# Define number of qubits:
n_qubits = 10

# Generate random parameters uniformly distributed between 0 and pi:
ex_params = np.random.uniform(0, np.pi, size=(2, n_qubits, 3))
print(ex_params)

# Parameter shape:
print("Parameter Shape:", ex_params.shape)

# Parameter length:
print("Parameter Length:", len(ex_params))

[[[1.91097246 1.01550529 0.76269756]
  [2.7081375  1.21722986 0.473691  ]
  [1.67221005 1.04698332 0.83598091]
  [3.04528226 2.43934445 2.95657263]
  [0.08133242 2.37507993 2.56402521]
  [2.16242043 0.90695172 1.22114814]
  [0.10047815 0.98840734 0.40175738]
  [3.03206013 2.0835179  0.02733872]
  [0.61894852 2.94742984 1.80715495]
  [0.79689259 1.89113454 0.97194361]]

 [[1.55046265 0.45417583 0.58851521]
  [3.01737425 1.10284837 2.74412284]
  [2.68225036 2.68456844 1.42131421]
  [0.63512856 0.41545463 2.56500947]
  [0.95741745 2.89408323 1.70424674]
  [0.29405411 1.65377332 1.35798604]
  [2.72274855 1.93542983 0.46403652]
  [1.01462262 0.35056723 1.91856101]
  [2.83735097 0.26953353 0.37314956]
  [0.08323459 1.61471193 0.5001151 ]]]
Parameter Shape: (2, 10, 3)
Parameter Length: 2


<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Gates_:</span>

In [22]:
# Custom General Rotation Gate for (variable) size/shape of weights and wires:
def G_Rot(weights, wire):
    """General Rotation Gate to Qubit."""
    qml.Rot(weights[0], weights[1], weights[2], wires=wire)

<span style="font-family: 'Computer Modern'; font-size: 14pt;">_Circuits_:</span>

In [23]:
# With general roations, no 'G_Rot':

def two_qubit_pooling(weights):
    
    # Important parameter values:
    n_qubits = 2
    n_weights = weights.shape[-1]
    
    # Apply Rotations/Gates before Measurement:
    for i in range(0, n_qubits, 2):
        # First set of Rotations (Qubit 1):
        qml.Rot(weights[0][i][0], weights[0][i][1], weights[0][i][2], wires=i)
        
        # First set of Rotations (Qubit 2):
        qml.Rot(weights[0][i+1][0], weights[0][i+1][1], weights[0][i+1][2], wires=i+1)
    
        # First set of CNOT Gates (Qubit 1):
        qml.CNOT(wires=[i, i+1])
    
        # Second set of Rotations (Qubit 1):
        qml.Rot(weights[1][i][0], weights[1][i][1], weights[1][i][2], wires=i)
    
        # Second set of Rotations (Qubit 2):
        qml.Rot(weights[1][i+1][0], weights[1][i+1][1], weights[1][i+1][2], wires=i+1)
    
        # Second set of CNOT Gates (Qubit 2):
        qml.CNOT(wires=[i, i+1])
    
        # Apply controlled rotations (CRot) directly:
        qml.CRot(weights[2][i][0], weights[2][i][1], weights[2][i][2], wires=[i, i+1])
    
    # NOTE: I commented out the PauliZ measure from the pooling function here, but this was strictly to be able to
    # use qml.draw below to visualize my results, which requires me to instantiate a circuit uses the layer 
    # function and returns a measure. When applied to the QCNN as a whole, this needs to be commented back in, or
    # else no dimensionality reduction will occur in the QCNN.
    
    # Perform PauliZ measure to every other Qubit:
    #for wire in range(0, n_qubits, 2):
    #    qml.expval(qml.PauliZ(wire))

<span style="font-family: 'Computer Modern'; font-weight: bold; font-size: 18pt;">DRAWINGS</span>

<span style="font-family: 'Computer Modern'; font-size: 14pt;">Drawing #1: _Original Pooling Layer w/ "Every Other Qubit" Measure (Not Used Currently)_:</span>

In [24]:
# Example Usage #1:

n_qubits = 2
dev = qml.device('default.qubit', wires=n_qubits)

# Circuit 1: Every Other Qubit
#------------------------------------------------------------------------------
@qml.qnode(dev)
def circuit1(weights):
    two_qubit_pooling(weights)
    return [qml.expval(qml.PauliZ(wire)) for wire in range(0, n_qubits, 2)]
#------------------------------------------------------------------------------

weights = np.random.random((3, n_qubits, 3))  # Example weights
print(f'Pooling Layer Results (2 Qubits) #1:')
print(circuit1(weights))

# Draw Circuit Layer:
unit_drawer1 = qml.draw(circuit1)
print(f' -> Pooling Layer with PauliZ Measure @ "Every Other Qubit":')
print(f'-------------------------------------------------------------------------------')
print(unit_drawer1(weights))
print(f'-------------------------------------------------------------------------------\n\n')

Pooling Layer Results (2 Qubits) #1:
[tensor(0.56057221, requires_grad=True)]
 -> Pooling Layer with PauliZ Measure @ "Every Other Qubit":
-------------------------------------------------------------------------------
0: ──Rot(0.02,0.87,0.72)─╭●──Rot(0.32,0.42,0.39)─╭●─╭●───────────────────┤  <Z>
1: ──Rot(0.67,0.30,0.97)─╰X──Rot(0.56,0.14,0.04)─╰X─╰Rot(0.64,0.99,0.91)─┤     
-------------------------------------------------------------------------------




<span style="font-family: 'Computer Modern'; font-size: 14pt;">Drawing #2: _Original Pooling Layer w/ "Second Qubit in Pair" Measure (Not Used Currently)_:</span>

In [25]:
# Circuit 2: Second Qubit of each Pair
@qml.qnode(dev)
#------------------------------------------------------------------------------
def circuit2(weights):
    two_qubit_pooling(weights)
    return [qml.expval(qml.PauliZ(wire)) for wire in range(1, n_qubits, 2)]
#------------------------------------------------------------------------------

print(f'Pooling Layer Results (2 Qubits) #2:')
print(circuit2(weights))

# Draw Circuit Layer:
unit_drawer2 = qml.draw(circuit2)
print(f' -> Pooling Layer with PauliZ Measure @ "Every Other Qubit":')
print(f'-------------------------------------------------------------------------------')
print(unit_drawer2(weights))
print(f'-------------------------------------------------------------------------------\n\n')

Pooling Layer Results (2 Qubits) #2:
[tensor(0.77711004, requires_grad=True)]
 -> Pooling Layer with PauliZ Measure @ "Every Other Qubit":
-------------------------------------------------------------------------------
0: ──Rot(0.02,0.87,0.72)─╭●──Rot(0.32,0.42,0.39)─╭●─╭●───────────────────┤     
1: ──Rot(0.67,0.30,0.97)─╰X──Rot(0.56,0.14,0.04)─╰X─╰Rot(0.64,0.99,0.91)─┤  <Z>
-------------------------------------------------------------------------------




<span style="font-family: 'Computer Modern'; font-size: 14pt;">Drawing #3: _New Convolutional Layer (Updated Original Convolutional Layer)_:</span>

In [26]:
# Convolutional Layer function for testing and drawing:

def quantum_conv_layer_test(weights):
    """
    Applies a quantum convolutional layer.
    """
    n_qubits = weights.shape[1]  # Number of qubits is second dimension of weights
    
    for i in range(0, n_qubits, 2):
        # First rotation on second qubit of pair:
        qml.RZ(-np.pi/2, wires=i + 1)
        
        # First set of rotations on first qubit of pair:
        qml.RZ(weights[0][i][0], wires=i)
        qml.RY(weights[0][i][1], wires=i + 1)
        
        # First CNOT gate:
        qml.CNOT(wires=[i, i + 1])
        
        # Second set of rotations on second qubit of pair:
        qml.RY(weights[0][i][2], wires=i + 1)
        
        # Second CNOT gate:
        qml.CNOT(wires=[i + 1, i])
        
        # Second rotation on first qubit of pair:
        qml.RZ(np.pi/2, wires=i)
        
# Example usage of the function within QNode:
#---------------------------------------------------------------------
n_qubits = 2
dev = qml.device('default.qubit', wires=n_qubits)

# Convolutional  Layer Test Drawing:
#---------------------------------------------------------------------
drawer_beta = qml.draw(quantum_conv_layer_test)
#---------------------------------------------------------------------

weights_b = np.random.random((3, n_qubits, 3))  # Example weights

print(f"Shape of Example Parameter: {weights_b.shape}")
print("------------------------")
print(r'Convolutional Layer Circuit (qml.draw):')
print(drawer_beta(weights_b))

Shape of Example Parameter: (3, 2, 3)
------------------------
Convolutional Layer Circuit (qml.draw):
1: ──RZ(-1.57)──RY(0.21)─╭X──RY(0.53)─╭●───────────┤  
0: ──RZ(0.18)────────────╰●───────────╰X──RZ(1.57)─┤  


<span style="font-family: 'Computer Modern'; font-size: 14pt;">Drawing #4: _New Pooling Layer (From GitHub Demo)_:</span>

In [27]:
# Pooling Layer function for testing and drawing:

def pennylane_pool_layer_test(weights):
    """
    Applies two-qubit pooling operation to inputted qubits, including controlled rotation based on 
    measurement.
    """
    n_qubits = weights.shape[1]  # Determine number of qubits from weights shape:

    # Generate Gell-Mann matrices for 2-D space (single qubit operators):
    pool_operators = generate_gell_mann(2)  # NEW FUNC
    
    # Loop over active qubits in pairs # FOR PAIRS OF QUBITS
    for i in range(0, n_qubits, 2):  # FOR PAIRS OF QUBITS
        q1 = i        # First qubit:
        q2 = i + 1    # Second qubit:

        # Extract weights for current qubit pair:
        new_weights = weights[:, i, :]  # Shape (a, 3):

        # Get convolutional operators V1 and V2 from pool operators and weights:
        v1 = get_conv_op(pool_operators, new_weights[0])  # Use first set of weights for V1  # NEW FUNC
        v2 = get_conv_op(pool_operators, new_weights[1])  # Use second set of weights for V2  # NEW FUNC

        # Apply Hadamard gate to first qubit:
        qml.Hadamard(wires=q1)
        
        # Apply first controlled unitary operation:
        qml.ControlledQubitUnitary(v1, control_wires=[q1], wires=[q2])
        
        # Apply Hadamard gate to second qubit:
        qml.Hadamard(wires=q2)
        
        # Apply second controlled unitary operation:
        qml.ControlledQubitUnitary(v2, control_wires=[q2], wires=[q1])

# Example usage of the function within QNode:
#-------------------------------------------------------------------------
n_qubits = 2
dev = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev)
def pool_test_circuit(weights):
    pennylane_pool_layer_test(weights)
    return qml.expval(qml.PauliZ(0))
#-------------------------------------------------------------------------

weights_a = np.random.random((2, n_qubits, 3))  # Example weights

# Pooling Layer Test Drawing:
#-------------------------------------------------------------------------
dev = qml.device("default.qubit", wires=n_qubits)
n_qubits = 2  # Number of qubits for pooling layer circuit drawing
drawer_alpha = qml.draw(pool_test_circuit)
#-------------------------------------------------------------------------

print(f"Shape of Example Parameter: {weights_a.shape}")
print("------------------------")
print(r'Pooling Layer Circuit (qml.draw):')
print(drawer_alpha(weights_a))

Shape of Example Parameter: (2, 2, 3)
------------------------
Pooling Layer Circuit (qml.draw):
0: ──H─╭●────────╭U(M1)─┤  <Z>
1: ────╰U(M0)──H─╰●─────┤     

M0 = 
[[ 0.24477152-0.29834938j -0.64635464-0.65825544j]
 [ 0.64635464-0.65825544j  0.24477152+0.29834938j]]
M1 = 
[[ 0.89355202-0.10930698j -0.12530012-0.41703314j]
 [ 0.12530012-0.41703314j  0.89355202+0.10930698j]]
